## Takes in initial datasets, runs LOOXV with Linear regressions (non-interacting and interacting terms) and caluclates correlation coefficients between predictions and average actual values

In [ ]:
#importing libraries
import pandas as pd
import math
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
import copy
#import xlsxwriter
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.svm import SVR
from sklearn.utils import shuffle

In [2]:
#reading in data files

#Add the path to where you have your files in lines 4 and 6
CompStartHere = pd.read_csv("ComputationalStartHere_AP.csv")

pairs = pd.read_csv("Parameter_Combinations_Recollect_082919_and_083019.csv")

#formatting data in readable mannner
CompStartHere.rename(columns={'Amplitude':'AmpMult','Modulation Index':'ModIn'},inplace=True)
pairs.rename(columns={'Modulation Index':'ModIn'},inplace=True)

CompStartHere.drop(['Process day','CellName','Slice'],axis=1,inplace=True)
pairs = pairs[['Cell Type','Duration','Frequency','AmpMult','ModIn']]

In [3]:
#combining both datasets
combined = pd.concat([CompStartHere,pairs]).reset_index(drop=True)
combined

,Cell Type,Duration,Frequency,AmpMult,ModIn
0,PV,20000,200,0.50,0.125364
1,Lhx6,100,20,1.25,-0.238095
2,Lhx6,30000,150,2.00,0.164969
3,Lhx6,500,50,1.00,-1.000000
4,Lhx6,500,10,2.50,-0.833333
...,...,...,...,...,...
398,Lhx6,1000,10,2.50,-0.030303
399,Lhx6,5000,150,1.00,0.202454
400,Lhx6,5000,50,0.50,-0.014493
401,Lhx6,10000,50,1.50,0.179487


In [4]:
#separate out into PV and Lhx6
combinedPV = combined.loc[combined['Cell Type']=='PV'].reset_index(drop=True)
combinedLhx6 = combined.loc[combined['Cell Type']=='Lhx6'].reset_index(drop=True)

In [5]:
#drop any NA values
combinedPV.dropna(inplace=True)
combinedLhx6.dropna(inplace=True)

In [6]:
#shuffle each separated set
combinedPV_Shuffled = combinedPV.sample(frac=1,random_state=2).reset_index(drop=True)
combinedLhx6_Shuffled = combinedLhx6.sample(frac=1,random_state=3).reset_index(drop=True)

In [7]:
#processing data

#reassess shapes of matrices
PVData = combinedPV_Shuffled
Lhx6Data = combinedLhx6_Shuffled

print(PVData.shape)
print(Lhx6Data.shape)
PVData = PVData.dropna()
Lhx6Data = Lhx6Data.dropna()
print(PVData.shape)
print(Lhx6Data.shape)


PVData.reset_index(inplace=True,drop=True)
Lhx6Data.reset_index(inplace=True,drop=True)



(205, 5)
(195, 5)
(205, 5)
(195, 5)


In [8]:
#For each point in PV or Lhx6, calculate the average ModIns and create a column
#in each PV and Lhx6 dataset with that average for each point

#add dummy columns of 0 to each dataset

PVData["Averages"] = np.zeros((205,1))
Lhx6Data["Averages"] = np.zeros((195,1))


PVData
        

,Cell Type,Duration,Frequency,AmpMult,ModIn,Averages
0,PV,5000,100,0.5,0.118943,0.0
1,PV,250,100,1.0,0.170732,0.0
2,PV,500,200,2.0,0.076923,0.0
3,PV,100,150,1.5,0.558442,0.0
4,PV,500,80,1.5,0.282051,0.0
...,...,...,...,...,...,...
200,PV,250,100,2.5,0.285141,0.0
201,PV,1000,10,2.0,0.333333,0.0
202,PV,250,100,2.0,0.138340,0.0
203,PV,50,150,2.0,0.739130,0.0


In [9]:
#calculate averages for each set of stimulation parameters for PV

setOfParamsPV = set()
for point in range(0,len(PVData)):
    #print(point)
    dur = PVData.loc[point,"Duration"]
    freq = PVData.loc[point,"Frequency"]
    ampmult = PVData.loc[point,"AmpMult"]
    if (dur,freq,ampmult) not in setOfParamsPV:
        setOfParamsPV.add((dur,freq,ampmult))
        
        replicates = PVData.loc[(PVData["Duration"]==dur)
                                &(PVData["Frequency"]==freq)
                                &(PVData["AmpMult"]==ampmult)]
        indices = list(PVData.loc[(PVData["Duration"]==dur)
                                &(PVData["Frequency"]==freq)
                                &(PVData["AmpMult"]==ampmult)].index)
        print(indices)
        average = replicates["ModIn"].mean()
        for index in indices:
            PVData.loc[index,"Averages"] = average

#####################################################################           
#calculate averages for each set of stimulation parameters for Lhx6            
            
setOfParamsLhx6 = set()
for point in range(0,len(Lhx6Data)):
    #print(point)
    dur = Lhx6Data.loc[point,"Duration"]
    freq = Lhx6Data.loc[point,"Frequency"]
    ampmult = Lhx6Data.loc[point,"AmpMult"]
    if (dur,freq,ampmult) not in setOfParamsLhx6:
        setOfParamsLhx6.add((dur,freq,ampmult))
        
        replicates = Lhx6Data.loc[(Lhx6Data["Duration"]==dur)
                                &(Lhx6Data["Frequency"]==freq)
                                &(Lhx6Data["AmpMult"]==ampmult)]
        indices = list(Lhx6Data.loc[(Lhx6Data["Duration"]==dur)
                                &(Lhx6Data["Frequency"]==freq)
                                &(Lhx6Data["AmpMult"]==ampmult)].index)
        average = replicates["ModIn"].mean()
        for index in indices:
            Lhx6Data.loc[index,"Averages"] = average
        

[0]
[1, 122, 154, 172]
[2, 26, 97, 180]
[3]
[4, 61, 120]
[5]
[6]
[7]
[8, 204]
[9, 76]
[10]
[11, 38, 52]
[12, 49]
[13]
[14]
[15, 129, 203]
[16, 65, 159, 177]
[17]
[18, 123]
[19]
[20, 95]
[21]
[22]
[23]
[24]
[25, 112, 118, 151, 170]
[27, 74]
[28, 48, 109]
[29, 130]
[30]
[31]
[32, 79, 113]
[33, 171]
[34]
[35]
[36, 160]
[37]
[39, 182]
[40]
[41]
[42]
[43]
[44, 175]
[45, 64, 106]
[46]
[47]
[50]
[51, 57, 67]
[53, 73, 87, 89, 181]
[54, 71, 88]
[55]
[56, 135, 185]
[58]
[59]
[60]
[62]
[63]
[66]
[68, 127]
[69]
[70, 165]
[72]
[75]
[77, 166]
[78]
[80]
[81]
[82]
[83]
[84]
[85]
[86, 146, 157]
[90]
[91, 164]
[92, 168]
[93]
[94, 152, 169, 202]
[96]
[98, 155]
[99, 128, 196]
[100, 163, 195]
[101, 124, 156]
[102, 132, 136, 149, 189]
[103, 143, 158, 193]
[104]
[105]
[107]
[108, 201]
[110]
[111, 150]
[114]
[115]
[116, 125]
[117]
[119]
[121, 167]
[126]
[131]
[133]
[134]
[137]
[138]
[139]
[140]
[141]
[142]
[144]
[145, 174]
[147]
[148]
[153]
[161]
[162]
[173]
[176]
[178]
[179]
[183]
[184]
[186, 199]
[187]
[188

In [10]:
#sanity checking
PVData.loc[1].to_frame()

,1
Cell Type,PV
Duration,250
Frequency,100
AmpMult,1
ModIn,0.170732
Averages,0.195103


In [11]:
#sanity checking
PVData.loc[122].to_frame()

,122
Cell Type,PV
Duration,250
Frequency,100
AmpMult,1
ModIn,0.11111
Averages,0.195103


In [12]:
#sanity checking
Lhx6Data

,Cell Type,Duration,Frequency,AmpMult,ModIn,Averages
0,Lhx6,50,100,2.00,-1.000000,-1.000000
1,Lhx6,5000,50,1.00,-1.000000,-0.757576
2,Lhx6,500,20,2.50,0.333333,0.333333
3,Lhx6,1000,10,0.50,0.037037,0.037037
4,Lhx6,1000,200,2.00,-0.875000,-0.832237
...,...,...,...,...,...,...
190,Lhx6,100,20,1.25,-0.238095,-0.238095
191,Lhx6,250,100,0.50,-0.466667,-0.351634
192,Lhx6,500,80,1.50,-0.466667,-0.453968
193,Lhx6,10000,50,1.50,0.175258,0.241844


In [13]:
#read out to CSV files for Gaussian Process Regressions in R
PVData.to_csv("firstTwoDataPV_shuffled_withAverages_notLogged.csv")
Lhx6Data.to_csv("firstTwoDataLhx6_shuffled_withAverages_notLogged.csv")

### IGNORE CODE BELOW: Have calculated averages, now training linear regressions, linear kernel and interacting terms, and comparing regression prediction results with the average modulation indices

In [15]:
#form training X's and Y's
#no "testing" since this is LOOXV

# XTrainPV = PVData[["Duration","Frequency","AmpMult"]]
# YTrainPV = PVData[["ModIn"]]

# XTrainLhx6 = Lhx6Data[["Duration","Frequency","AmpMult"]]
# YTrainLhx6 = Lhx6Data[["ModIn"]]


In [16]:
# #log the duration values for PV

# XTrainPVLogged = XTrainPV.assign(Duration=np.log2(XTrainPV["Duration"]))

# XTrainPVLogged.reset_index(inplace=True,drop=True)

# XTrainPVLogged


,Duration,Frequency,AmpMult
0,12.287712,100,0.5
1,7.965784,100,1.0
2,8.965784,200,2.0
3,6.643856,150,1.5
4,8.965784,80,1.5
...,...,...,...
200,7.965784,100,2.5
201,9.965784,10,2.0
202,7.965784,100,2.0
203,5.643856,150,2.0


In [17]:
# #sanity check reset index
# XTrainPV.reset_index(inplace=True,drop=True)
# YTrainPV.reset_index(inplace=True,drop=True)

# XTrainPV

,Duration,Frequency,AmpMult
0,5000,100,0.5
1,250,100,1.0
2,500,200,2.0
3,100,150,1.5
4,500,80,1.5
...,...,...,...
200,250,100,2.5
201,1000,10,2.0
202,250,100,2.0
203,50,150,2.0


In [18]:
# #log the duration for Lhx6 data
# XTrainLhx6Logged = XTrainLhx6.assign(Duration=np.log2(XTrainLhx6["Duration"]))
# XTrainLhx6Logged.reset_index(inplace=True,drop=True)
# XTrainLhx6.reset_index(inplace=True,drop=True)
# YTrainLhx6.reset_index(inplace=True,drop=True)

# XTrainLhx6


,Duration,Frequency,AmpMult
0,50,100,2.00
1,5000,50,1.00
2,500,20,2.50
3,1000,10,0.50
4,1000,200,2.00
...,...,...,...
190,100,20,1.25
191,250,100,0.50
192,500,80,1.50
193,10000,50,1.50


In [19]:
# #train looxv linear regression for both data sets

# #first do PV dataset
# #XTrainPV has normal duration, XTrainPVLogged has logged duration
# predictedPV_Vals = []
# actualPV_Vals = []
# for idx in list(XTrainPV.index):
#     print(idx)
#     heldOutPointX = XTrainPV.iloc[[idx]]
#     heldOutPointY = YTrainPV.iloc[[idx]]
#     heldOutDur = heldOutPointX["Duration"].item()
#     heldOutFreq = heldOutPointX["Frequency"].item()
#     heldOutAmp = heldOutPointX["AmpMult"].item()
    
#     #hold out all instances of the same parameters
#     delIndices = XTrainPV.loc[(XTrainPV["Duration"]==heldOutDur) &
#                                 (XTrainPV["Frequency"]==heldOutFreq) &
#                                 (XTrainPV["AmpMult"]==heldOutAmp)].index.to_list()
#     print(delIndices)
    
#     #Reassign newTrainX to XTrainPVLogged.drop so XTrainPVLogged isn't affected
#     newTrainX = XTrainPVLogged.drop(labels=delIndices)
#     print(len(newTrainX))
#     newTrainY = YTrainPV.drop(labels=delIndices)

#     regPV = LinearRegression().fit(newTrainX,newTrainY)
    
#     heldOutPointX["Duration"] = np.log2(heldOutDur)
#     predictedPV_Vals.append(regPV.predict(heldOutPointX.to_numpy()))
#     actualPV_Vals.append(heldOutPointY.to_numpy())

# #store predicted PV values and the actual PV values
# predictedPV_Vals = np.asarray(predictedPV_Vals).flatten()
# actualPV_Vals = np.asarray(actualPV_Vals).flatten()




0
[0]
204
1
[1, 122, 154, 172]
201
2
[2, 26, 97, 180]
201
3
[3]
204
4
[4, 61, 120]
202
5
[5]
204
6
[6]
204
7
[7]
204
8
[8, 204]
203
9
[9, 76]
203
10
[10]
204
11
[11, 38, 52]
202
12
[12, 49]
203
13
[13]
204
14
[14]
204
15
[15, 129, 203]
202
16
[16, 65, 159, 177]
201
17
[17]
204
18
[18, 123]
203
19
[19]
204
20


/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/noelletoong/anaconda3/lib/py

[20, 95]
203
21
[21]
204
22
[22]
204
23
[23]
204
24
[24]
204
25
[25, 112, 118, 151, 170]
200
26
[2, 26, 97, 180]
201
27
[27, 74]
203
28
[28, 48, 109]
202
29
[29, 130]
203
30
[30]
204
31
[31]
204
32
[32, 79, 113]
202
33
[33, 171]
203
34
[34]
204
35
[35]
204
36
[36, 160]
203
37
[37]
204
38
[11, 38, 52]
202
39
[39, 182]
203
40
[40]
204
41
[41]
204
42
[42]
204
43
[43]
204
44
[44, 175]
203
45


/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/noelletoong/anaconda3/lib/py

[45, 64, 106]
202
46
[46]
204
47
[47]
204
48
[28, 48, 109]
202
49
[12, 49]
203
50
[50]
204
51
[51, 57, 67]
202
52
[11, 38, 52]
202
53
[53, 73, 87, 89, 181]
200
54
[54, 71, 88]
202
55
[55]
204
56
[56, 135, 185]
202
57
[51, 57, 67]
202
58
[58]
204
59
[59]
204
60
[60]
204
61
[4, 61, 120]
202
62
[62]
204
63
[63]
204
64
[45, 64, 106]
202
65
[16, 65, 159, 177]
201
66
[66]
204
67
[51, 57, 67]
202
68
[68, 127]
203
69
[69]
204
70
[70, 165]
203
71
[54, 71, 88]
202


...

/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/py


204
163
[100, 163, 195]
202
164
[91, 164]
203
165
[70, 165]
203
166
[77, 166]
203
167
[121, 167]
203
168
[92, 168]
203
169
[94, 152, 169, 202]
201
170
[25, 112, 118, 151, 170]
200
171
[33, 171]
203
172
[1, 122, 154, 172]
201
173
[173]
204
174
[145, 174]
203
175
[44, 175]
203
176
[176]
204
177
[16, 65, 159, 177]
201
178
[178]
204
179
[179]
204
180
[2, 26, 97, 180]
201
181
[53, 73, 87, 89, 181]
200
182
[39, 182]
203
183
[183]
204
184
[184]
204
185
[56, 135, 185]
202
186
[186, 199]
203
187
[187]
204
188
[188]
204
189
[102, 132, 136, 149, 189]
200
190
[190]
204
191
[191]
204


/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/py

192
[192]
204
193
[103, 143, 158, 193]
201
194
[194]
204
195
[100, 163, 195]
202
196
[99, 128, 196]
202
197
[197]
204
198
[198]
204
199
[186, 199]
203
200
[200]
204
201
[108, 201]
203
202
[94, 152, 169, 202]
201
203
[15, 129, 203]
202
204
[8, 204]
203


/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()
/Users/noelletoong/anaconda3/lib/py

In [20]:
# #Add the predictions as a column to the dataframe
# #Drop the duplicate parameter points
# PVData["predictions_SingleLR"] = predictedPV_Vals
# reducedPVData_SingleLR = PVData.drop_duplicates(subset=["Duration","Frequency","AmpMult"]).reset_index(drop=True)
# reducedPVData_SingleLR

,Cell Type,Duration,Frequency,AmpMult,ModIn,Averages,predictions_SingleLR
0,PV,5000,100,0.50,0.118943,0.118943,0.182464
1,PV,250,100,1.00,0.170732,0.195103,0.245019
2,PV,500,200,2.00,0.076923,0.228159,0.342745
3,PV,100,150,1.50,0.558442,0.558442,0.310557
4,PV,500,80,1.50,0.282051,0.261553,0.206143
...,...,...,...,...,...,...,...
124,PV,5000,200,1.25,0.084065,0.084065,0.289872
125,PV,500,100,0.50,0.648590,0.648590,0.222873
126,PV,100,80,1.00,0.219512,0.219512,0.241696
127,PV,500,20,1.50,0.037037,0.037037,0.145414


In [21]:
# #Calculate the Pearson and Spearman coefficients between the average actual responses and 
# print("Pearson's correlation value and p-value for LOOXV PV data:")
# print(pearsonr(reducedPVData_SingleLR["predictions_SingleLR"],reducedPVData_SingleLR["Averages"]))
# print(spearmanr(reducedPVData_SingleLR["predictions_SingleLR"],reducedPVData_SingleLR["Averages"]))
# print("\n")

Pearson's correlation value and p-value for LOOXV PV data:
(0.2674676839288977, 0.0021817210505236605)
SpearmanrResult(correlation=0.3670225849731664, pvalue=1.8833850404439448e-05)




In [22]:
# #next do Lhx6 dataset LOOXV linear regression, linear kernel
# #same concept as PV dataset above

# predictedLhx6_Vals = []
# actualLhx6_Vals = []
# for idx in range(0,len(XTrainLhx6)):
#     print(idx)
#     heldOutPointX = XTrainLhx6.iloc[[idx]]
#     heldOutPointY = YTrainLhx6.iloc[[idx]]
#     #print(heldOutPointX["AmpMult"].item())
#     heldOutDur = heldOutPointX["Duration"].item()
#     heldOutFreq = heldOutPointX["Frequency"].item()
#     heldOutAmp = heldOutPointX["AmpMult"].item()
    
#     delIndices = XTrainLhx6.loc[(XTrainLhx6["Duration"]==heldOutDur) &
#                                 (XTrainLhx6["Frequency"]==heldOutFreq) &
#                                 (XTrainLhx6["AmpMult"]==heldOutAmp)].index.to_list()
#     print(delIndices)
    
#     newTrainX = XTrainLhx6Logged.drop(labels=delIndices)
#     print(len(newTrainX))
#     newTrainY = YTrainLhx6.drop(labels=delIndices)

#     regLhx6 = LinearRegression().fit(newTrainX,newTrainY)
    
#     heldOutPointX["Duration"] = np.log2(heldOutDur)
#     predictedLhx6_Vals.append(regLhx6.predict(heldOutPointX.to_numpy()))
#     actualLhx6_Vals.append(heldOutPointY.to_numpy())

# predictedLhx6_Vals = np.asarray(predictedLhx6_Vals).flatten()
# actualLhx6_Vals = np.asarray(actualLhx6_Vals).flatten()



0
[0]
194
1
[1, 8]
193
2
[2]
194
3
[3]
194
4
[4, 19]
193
5
[5, 9, 38, 180]
191
6
[6, 22]
193
7
[7]
194
8
[1, 8]
193
9
[5, 9, 38, 180]
191
10
[10, 84]
193
11
[11]
194
12
[12, 53]
193
13
[13, 21, 67, 102, 194]
190
14
[14]
194
15
[15, 107, 185]
192
16
[16, 133]
193
17
[17]
194
18
[18, 63, 179, 188]
191
19
[4, 19]
193
20
[20, 112, 115, 163]
191
21
[13, 21, 67, 102, 194]
190
22
[6, 22]
193
23
[23]
194
24
[24]
194
25
[25]
194
26
[26]
194
27
[27, 49, 76, 89]
191
28
[28]
194
29
[29, 95]
193
30
[30]
194
31
[31]

/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: `item` has been deprecated and will be removed in a future version
  if __name__ == '__main__':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: `item` has been deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-


194
32
[32]
194
33
[33, 124, 127]
192
34
[34, 52, 135]
192
35
[35]
194
36
[36, 45]
193
37
[37, 93]
193
38
[5, 9, 38, 180]
191
39
[39, 96]
193
40
[40, 162, 193]
192
41
[41, 155]
193
42
[42]
194
43
[43]
194
44
[44]
194
45
[36, 45]
193
46
[46]
194
47
[47, 74]
193
48
[48]
194
49
[27, 49, 76, 89]
191
50
[50, 79, 187]
192
51
[51, 90, 103, 126]
191
52
[34, 52, 135]
192
53
[12, 53]
193
54
[54, 152, 174]
192
55


/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: `item` has been deprecated and will be removed in a future version
  if __name__ == '__main__':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: `item` has been deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_pa

[55, 144, 172]
192
56
[56]
194
57
[57]
194
58
[58]
194
59
[59]
194
60
[60]
194
61
[61]
194
62
[62]
194
63
[18, 63, 179, 188]
191
64
[64, 143]
193
65
[65, 92, 139]
192
66
[66]
194
67
[13, 21, 67, 102, 194]
190
68
[68, 149, 170, 177]
191
69
[69]
194
70
[70, 105]
193
71
[71]

...

/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: `item` has been deprecated and will be removed in a future version
  if __name__ == '__main__':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: `item` has been deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_pa

139
[65, 92, 139]
192
140
[73, 140, 175]
192
141
[141]
194
142
[91, 142]
193
143
[64, 143]
193
144
[55, 144, 172]
192
145
[145]
194
146
[146]
194
147
[100, 147, 184]
192
148
[148]
194
149
[68, 149, 170, 177]
191
150
[122, 150]
193
151
[151]
194
152
[54, 152, 174]
192
153
[104, 128, 153, 154]
191
154
[104, 128, 153, 154]
191
155
[41, 155]
193
156
[156]
194
157
[157]
194
158
[158]
194
159
[159]
194
160
[160]
194
161
[161]
194
162
[40, 162, 193]
192
163
[20, 112, 115, 163]
191
164
[106, 164]

/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: `item` has been deprecated and will be removed in a future version
  if __name__ == '__main__':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: `item` has been deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_pa


193
165
[165]
194
166
[166]
194
167
[167]
194
168
[168]
194
169
[169]
194
170
[68, 149, 170, 177]
191
171
[171]
194
172
[55, 144, 172]
192
173
[173]
194
174
[54, 152, 174]
192
175
[73, 140, 175]
192
176
[176]
194
177
[68, 149, 170, 177]
191
178
[121, 178]
193
179
[18, 63, 179, 188]
191
180
[5, 9, 38, 180]
191
181
[86, 116, 134, 181]
191
182
[129, 182]
193
183
[183]
194
184
[100, 147, 184]
192
185
[15, 107, 185]
192
186
[83, 186]
193
187
[50, 79, 187]
192
188
[18, 63, 179, 188]
191
189
[189]
194
190
[190]
194
191
[88, 117, 191]
192
192
[109, 131, 192]
192
193
[40, 162, 193]
192
194
[13, 21, 67, 102, 194]
190


/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: `item` has been deprecated and will be removed in a future version
  if __name__ == '__main__':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: `item` has been deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [23]:
# #Add predictions as new column and drop the duplicate parameter settings for Lhx6 data
# Lhx6Data["predictions_SingleLR"] = predictedLhx6_Vals
# reducedLhx6Data_SingleLR = Lhx6Data.drop_duplicates(subset=["Duration","Frequency","AmpMult"]).reset_index(drop=True)
# reducedLhx6Data_SingleLR

,Cell Type,Duration,Frequency,AmpMult,ModIn,Averages,predictions_SingleLR
0,Lhx6,50,100,2.00,-1.000000,-1.000000,-0.723285
1,Lhx6,5000,50,1.00,-1.000000,-0.757576,-0.138198
2,Lhx6,500,20,2.50,0.333333,0.333333,-0.597218
3,Lhx6,1000,10,0.50,0.037037,0.037037,-0.185405
4,Lhx6,1000,200,2.00,-0.875000,-0.832237,-0.529741
...,...,...,...,...,...,...,...
114,Lhx6,50,10,1.50,0.311475,0.311475,-0.614443
115,Lhx6,30000,100,1.00,-0.250000,-0.250000,-0.034224
116,Lhx6,1000,150,2.50,0.098039,0.098039,-0.614574
117,Lhx6,250,200,1.00,-1.000000,-1.000000,-0.478014


In [24]:
# #Calculate the Pearson and Spearman coefficients
# print("Pearson's correlation value and p-value for LOOXV Lhx6 data:")
# print(pearsonr(reducedLhx6Data_SingleLR["predictions_SingleLR"],reducedLhx6Data_SingleLR["Averages"]))
# print(spearmanr(reducedLhx6Data_SingleLR["predictions_SingleLR"],reducedLhx6Data_SingleLR["Averages"]))

Pearson's correlation value and p-value for LOOXV Lhx6 data:
(0.3320719662505216, 0.0002246885337438786)
SpearmanrResult(correlation=0.3241823941180376, pvalue=0.0003222497991296858)


## Repeat process above with interaction terms

In [26]:
# #a+b+c+(a*b)+(a*c)+(b*c)+(a*b*c)
# #create X_df with 7 columns
# #use same Y
# #for PV data

# XTrainPVIntx = pd.DataFrame()
# XTrainPVIntx["Duration"] = np.log2(XTrainPV["Duration"])
# XTrainPVIntx["Frequency"] = XTrainPV["Frequency"]
# XTrainPVIntx["AmpMult"] = XTrainPV["AmpMult"]
# XTrainPVIntx["Dur*Freq"] = np.log2(XTrainPV["Duration"])*XTrainPV["Frequency"]
# XTrainPVIntx["Dur*AmpMult"] = np.log2(XTrainPV["Duration"])*XTrainPV["AmpMult"]
# XTrainPVIntx["Freq*AmpMult"] = XTrainPV["Frequency"]*XTrainPV["AmpMult"]
# XTrainPVIntx["Dur*Freq*AmpMult"] = np.log2(XTrainPV["Duration"])*XTrainPV["Frequency"]*XTrainPV["AmpMult"]
# XTrainPVIntx



,Duration,Frequency,AmpMult,Dur*Freq,Dur*AmpMult,Freq*AmpMult,Dur*Freq*AmpMult
0,12.287712,100,0.5,1228.771238,6.143856,50.0,614.385619
1,7.965784,100,1.0,796.578428,7.965784,100.0,796.578428
2,8.965784,200,2.0,1793.156857,17.931569,400.0,3586.313714
3,6.643856,150,1.5,996.578428,9.965784,225.0,1494.867643
4,8.965784,80,1.5,717.262743,13.448676,120.0,1075.894114
...,...,...,...,...,...,...,...
200,7.965784,100,2.5,796.578428,19.914461,250.0,1991.446071
201,9.965784,10,2.0,99.657843,19.931569,20.0,199.315686
202,7.965784,100,2.0,796.578428,15.931569,200.0,1593.156857
203,5.643856,150,2.0,846.578428,11.287712,300.0,1693.156857


In [27]:

# #create interaction terms for Lhx6 data
# XTrainLhx6Intx = pd.DataFrame()
# XTrainLhx6Intx["Duration"] = np.log2(XTrainLhx6["Duration"])
# XTrainLhx6Intx["Frequency"] = XTrainLhx6["Frequency"]
# XTrainLhx6Intx["AmpMult"] = XTrainLhx6["AmpMult"]
# XTrainLhx6Intx["Dur*Freq"] = np.log2(XTrainLhx6["Duration"])*XTrainLhx6["Frequency"]
# XTrainLhx6Intx["Dur*AmpMult"] = np.log2(XTrainLhx6["Duration"])*XTrainLhx6["AmpMult"]
# XTrainLhx6Intx["Freq*AmpMult"] = XTrainLhx6["Frequency"]*XTrainLhx6["AmpMult"]
# XTrainLhx6Intx["Dur*Freq*AmpMult"] = np.log2(XTrainLhx6["Duration"])*XTrainLhx6["Frequency"]*XTrainLhx6["AmpMult"]
# XTrainLhx6Intx


,Duration,Frequency,AmpMult,Dur*Freq,Dur*AmpMult,Freq*AmpMult,Dur*Freq*AmpMult
0,5.643856,100,2.00,564.385619,11.287712,200.0,1128.771238
1,12.287712,50,1.00,614.385619,12.287712,50.0,614.385619
2,8.965784,20,2.50,179.315686,22.414461,50.0,448.289214
3,9.965784,10,0.50,99.657843,4.982892,5.0,49.828921
4,9.965784,200,2.00,1993.156857,19.931569,400.0,3986.313714
...,...,...,...,...,...,...,...
190,6.643856,20,1.25,132.877124,8.304820,25.0,166.096405
191,7.965784,100,0.50,796.578428,3.982892,50.0,398.289214
192,8.965784,80,1.50,717.262743,13.448676,120.0,1075.894114
193,13.287712,50,1.50,664.385619,19.931569,75.0,996.578428


In [28]:
# #perform looxv for both PV and Lhx6 datasets, as in above

# predictedPV_ValsIntx = []
# actualPV_Vals = []
# for idx in list(XTrainPVIntx.index):
#     print(idx)
#     #this has the logged duration
#     heldOutPointXLogged = XTrainPVIntx.iloc[[idx]]
#     heldOutPointY = YTrainPV.iloc[[idx]]
    
#     #this does not have the logged duration, to be used to take out replicates
#     #do NOT use the logged duration to take out replicates
#     heldOutPointX = XTrainPV.iloc[[idx]]

#     heldOutDur = heldOutPointX["Duration"].item()
#     heldOutFreq = heldOutPointX["Frequency"].item()
#     heldOutAmp = heldOutPointX["AmpMult"].item()
#     print(heldOutDur)
#     delIndices = XTrainPV.loc[(XTrainPV["Duration"]==heldOutDur) &
#                                 (XTrainPV["Frequency"]==heldOutFreq) &
#                                 (XTrainPV["AmpMult"]==heldOutAmp)].index.to_list()
#     print(delIndices)
 
#     newTrainX = XTrainPVIntx.drop(labels=delIndices)
#     print(len(newTrainX))
#     newTrainY = YTrainPV.drop(labels=delIndices)

#     regPV = LinearRegression().fit(newTrainX,newTrainY)
    
#     #print(heldOutPointXLogged)
#     predictedPV_ValsIntx.append(regPV.predict(heldOutPointXLogged.to_numpy()))
#     actualPV_Vals.append(heldOutPointY.to_numpy())

# predictedPV_ValsIntx = np.asarray(predictedPV_ValsIntx).flatten()
# actualPV_Vals = np.asarray(actualPV_Vals).flatten()





0
5000
[0]
204
1
250
[1, 122, 154, 172]
201
2
500
[2, 26, 97, 180]
201
3
100
[3]
204
4
500
[4, 61, 120]
202
5
10000
[5]
204
6
500
[6]
204
7
50
[7]
204
8
250
[8, 204]
203
9
1000
[9, 76]
203
10
20000
[10]
204
11
100
[11, 38, 52]
202
12
250
[12, 49]
203
13
500
[13]
204
14
100
[14]
204
15
50
[15, 129, 203]
202
16
10000
[16, 65, 159, 177]
201
17
5000
[17]
204
18
50
[18, 123]
203
19
1000
[19]
204
20
20000
[20, 95]
203
21
30000
[21]
204
22
500
[22]
204
23
50
[23]
204
24
10000
[24]
204
25
250
[25, 112, 118, 151, 170]
200
26
500
[2, 26, 97, 180]
201
27
1000
[27, 74]
203
28
500
[28, 48, 109]
202
29
50
[29, 130]
203
30
100
[30]
204
31

/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: `item` has been deprecated and will be removed in a future version
  from ipykernel import kernelapp as app
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: `item` has been deprecated and will be removed in a future version
  app.launch_new_instance()
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: `item` has been deprecated and will be removed in a future version
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: `item` has been deprecated and will be removed in a future version
  from ipykernel import kernelapp as app
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: `item` has been deprecated and will be removed in a future version
  app.launch_new_instance()
/Users/noelletoong/anaconda3/lib/pyth


250
[31]
204
32
500
[32, 79, 113]
202
33
500
[33, 171]
203
34
20000
[34]
204
35
20000
[35]
204
36
30000
[36, 160]
203
37
30000
[37]
204
38
100
[11, 38, 52]
202
39
500
[39, 182]
203
40
30000
[40]
204
41
1000
[41]
204
42
10000
[42]
204
43
100
[43]
204
44
1000
[44, 175]
203
45
1000
[45, 64, 106]
202
46
10000
[46]
204
47
1000
[47]
204
48
500
[28, 48, 109]
202
49
250
[12, 49]
203
50
500
[50]
204
51
500
[51, 57, 67]
202
52
100
[11, 38, 52]
202
53
250
[53, 73, 87, 89, 181]
200
54
30000
[54, 71, 88]
202
55
250
[55]
204
56
100
[56, 135, 185]
202
57
500
[51, 57, 67]
202
58
5000
[58]
204
59
250
[59]
204
60
10000
[60]
204
61
500
[4, 61, 120]
202
62
50
[62]
204
63
500
[63]

/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: `item` has been deprecated and will be removed in a future version
  from ipykernel import kernelapp as app
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: `item` has been deprecated and will be removed in a future version
  app.launch_new_instance()
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: `item` has been deprecated and will be removed in a future version
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: `item` has been deprecated and will be removed in a future version
  from ipykernel import kernelapp as app
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: `item` has been deprecated and will be removed in a future version
  app.launch_new_instance()
/Users/noelletoong/anaconda3/lib/pyth


204
64
1000
[45, 64, 106]
202
65
10000
[16, 65, 159, 177]
201
66
1000
[66]
204
67
500
[51, 57, 67]
202
68
50
[68, 127]
203
69
50
[69]
204
70
500
[70, 165]
203
71
30000
[54, 71, 88]
202
72
500
[72]
204
73
250
[53, 73, 87, 89, 181]
200
74
1000
[27, 74]
203
75
30000
[75]
204
76
1000
[9, 76]
203
77
10000
[77, 166]
203
78
30000
[78]
204
79
500
[32, 79, 113]
202
80
50
[80]
204
81
50
[81]
204
82
250
[82]
204
83
20000
[83]
204
84
500
[84]
204
85
500
[85]
204
86
500
[86, 146, 157]
202
87
250
[53, 73, 87, 89, 181]
200
88
30000
[54, 71, 88]
202
89
250
[53, 73, 87, 89, 181]
200
90
1000
[90]
204


...

/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: `item` has been deprecated and will be removed in a future version
  from ipykernel import kernelapp as app
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: `item` has been deprecated and will be removed in a future version
  app.launch_new_instance()
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: `item` has been deprecated and will be removed in a future version
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: `item` has been deprecated and will be removed in a future version
  from ipykernel import kernelapp as app
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: `item` has been deprecated and will be removed in a future version
  app.launch_new_instance()
/Users/noelletoong/anaconda3/lib/pyth

162
100
[162]
204
163
10000
[100, 163, 195]
202
164
250
[91, 164]
203
165
500
[70, 165]
203
166
10000
[77, 166]
203
167
30000
[121, 167]
203
168
5000
[92, 168]
203
169
250
[94, 152, 169, 202]
201
170
250
[25, 112, 118, 151, 170]
200
171
500
[33, 171]
203
172
250
[1, 122, 154, 172]
201
173
30000
[173]
204
174
500
[145, 174]
203
175
1000
[44, 175]
203
176
10000
[176]
204
177
10000
[16, 65, 159, 177]
201
178
50
[178]
204
179
1000
[179]
204
180
500
[2, 26, 97, 180]
201
181
250
[53, 73, 87, 89, 181]
200
182
500
[39, 182]
203
183
10000
[183]
204
184
10000
[184]
204
185
100
[56, 135, 185]
202
186
250
[186, 199]
203
187
500
[187]
204
188
500
[188]
204
189
1000
[102, 132, 136, 149, 189]
200
190

/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: `item` has been deprecated and will be removed in a future version
  from ipykernel import kernelapp as app
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: `item` has been deprecated and will be removed in a future version
  app.launch_new_instance()
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: `item` has been deprecated and will be removed in a future version
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: `item` has been deprecated and will be removed in a future version
  from ipykernel import kernelapp as app
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: `item` has been deprecated and will be removed in a future version
  app.launch_new_instance()
/Users/noelletoong/anaconda3/lib/pyth


50
[190]
204
191
100
[191]
204
192
5000
[192]
204
193
1000
[103, 143, 158, 193]
201
194
500
[194]
204
195
10000
[100, 163, 195]
202
196
5000
[99, 128, 196]
202
197
100
[197]
204
198
500
[198]
204
199
250
[186, 199]
203
200
250
[200]
204
201
1000
[108, 201]
203
202
250
[94, 152, 169, 202]
201
203
50
[15, 129, 203]
202
204
250
[8, 204]
203


/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: `item` has been deprecated and will be removed in a future version
  from ipykernel import kernelapp as app
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: `item` has been deprecated and will be removed in a future version
  app.launch_new_instance()
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: `item` has been deprecated and will be removed in a future version
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: `item` has been deprecated and will be removed in a future version
  from ipykernel import kernelapp as app
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: `item` has been deprecated and will be removed in a future version
  app.launch_new_instance()
/Users/noelletoong/anaconda3/lib/pyth

In [29]:
# #Add interaction terms predictions as a new column
# PVData["predictions_IntxLR"] = predictedPV_ValsIntx
# PVData

,Cell Type,Duration,Frequency,AmpMult,ModIn,Averages,predictions_SingleLR,predictions_IntxLR
0,PV,5000,100,0.5,0.118943,0.118943,0.182464,0.162483
1,PV,250,100,1.0,0.170732,0.195103,0.245019,0.258981
2,PV,500,200,2.0,0.076923,0.228159,0.342745,0.353464
3,PV,100,150,1.5,0.558442,0.558442,0.310557,0.350642
4,PV,500,80,1.5,0.282051,0.261553,0.206143,0.200506
...,...,...,...,...,...,...,...,...
200,PV,250,100,2.5,0.285141,0.285141,0.240216,0.213134
201,PV,1000,10,2.0,0.333333,0.155039,0.117237,0.124445
202,PV,250,100,2.0,0.138340,0.241831,0.241814,0.228174
203,PV,50,150,2.0,0.739130,0.703801,0.300958,0.300662


In [30]:
# #Get rid of duplicate parameter settings
# reducedPVData_IntxLR = PVData.drop_duplicates(subset=["Duration","Frequency","AmpMult"]).reset_index(drop=True)
# reducedPVData_IntxLR

,Cell Type,Duration,Frequency,AmpMult,ModIn,Averages,predictions_SingleLR,predictions_IntxLR
0,PV,5000,100,0.50,0.118943,0.118943,0.182464,0.162483
1,PV,250,100,1.00,0.170732,0.195103,0.245019,0.258981
2,PV,500,200,2.00,0.076923,0.228159,0.342745,0.353464
3,PV,100,150,1.50,0.558442,0.558442,0.310557,0.350642
4,PV,500,80,1.50,0.282051,0.261553,0.206143,0.200506
...,...,...,...,...,...,...,...,...
124,PV,5000,200,1.25,0.084065,0.084065,0.289872,0.228126
125,PV,500,100,0.50,0.648590,0.648590,0.222873,0.238115
126,PV,100,80,1.00,0.219512,0.219512,0.241696,0.251129
127,PV,500,20,1.50,0.037037,0.037037,0.145414,0.128910


In [31]:
# #Calculate Pearson and Spearman correlation coefficients for PV data
# print("Pearson's correlation value and p-value for LOOXV PV data interaction terms:")
# print(pearsonr(reducedPVData_IntxLR["predictions_IntxLR"],reducedPVData_IntxLR["Averages"]))
# print(spearmanr(reducedPVData_IntxLR["predictions_IntxLR"],reducedPVData_IntxLR["Averages"]))
# print("\n")

Pearson's correlation value and p-value for LOOXV PV data interaction terms:
(0.2414950926041703, 0.005831817963851644)
SpearmanrResult(correlation=0.40029628801431133, pvalue=2.5977413990934185e-06)




In [32]:
# #perform looxv for Lhx6 dataset, as in above

# predictedLhx6_ValsIntx = []
# actualLhx6_Vals = []
# for idx in list(XTrainLhx6Intx.index):

#     heldOutPointXLogged = XTrainLhx6Intx.iloc[[idx]]
#     heldOutPointY = YTrainLhx6.iloc[[idx]]

#     heldOutPointX = XTrainLhx6.iloc[[idx]]
    
#     heldOutDur = heldOutPointX["Duration"].item()
#     heldOutFreq = heldOutPointX["Frequency"].item()
#     heldOutAmp = heldOutPointX["AmpMult"].item()
    
#     delIndices = XTrainLhx6.loc[(XTrainLhx6["Duration"]==heldOutDur) &
#                                 (XTrainLhx6["Frequency"]==heldOutFreq) &
#                                 (XTrainLhx6["AmpMult"]==heldOutAmp)].index.to_list()
 
    
#     newTrainX = XTrainLhx6Intx.drop(labels=delIndices)
#     newTrainY = YTrainLhx6.drop(labels=delIndices)
#     assert(len(newTrainX)==len(XTrainLhx6Intx)-len(delIndices))
#     regLhx6 = LinearRegression().fit(newTrainX,newTrainY)
    
#     predictedLhx6_ValsIntx.append(regLhx6.predict(heldOutPointXLogged.to_numpy()))
#     actualLhx6_Vals.append(heldOutPointY.to_numpy())

# predictedLhx6_ValsIntx = np.asarray(predictedLhx6_ValsIntx).flatten()
# actualLhx6_Vals = np.asarray(actualLhx6_Vals).flatten()



/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: Futur

/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has 

/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has 

/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: Futur

/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has 

/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: `item` has been deprecated and will be removed in a future version
  
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: `item` has been deprecated and will be removed in a future version
  del sys.path[0]
/Users/noelletoong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: Futur

In [33]:
# #Add new column for Lhx6 interaction term predictions
# Lhx6Data["predictions_IntxLR"] = predictedLhx6_ValsIntx
# Lhx6Data

,Cell Type,Duration,Frequency,AmpMult,ModIn,Averages,predictions_SingleLR,predictions_IntxLR
0,Lhx6,50,100,2.00,-1.000000,-1.000000,-0.723285,-0.795850
1,Lhx6,5000,50,1.00,-1.000000,-0.757576,-0.138198,-0.180481
2,Lhx6,500,20,2.50,0.333333,0.333333,-0.597218,-0.597172
3,Lhx6,1000,10,0.50,0.037037,0.037037,-0.185405,-0.217995
4,Lhx6,1000,200,2.00,-0.875000,-0.832237,-0.529741,-0.563946
...,...,...,...,...,...,...,...,...
190,Lhx6,100,20,1.25,-0.238095,-0.238095,-0.503818,-0.449343
191,Lhx6,250,100,0.50,-0.466667,-0.351634,-0.347303,-0.283721
192,Lhx6,500,80,1.50,-0.466667,-0.453968,-0.445070,-0.444206
193,Lhx6,10000,50,1.50,0.175258,0.241844,-0.199992,-0.226680


In [34]:
# #Get rid of duplicate settings
# reducedLhx6Data_IntxLR = Lhx6Data.drop_duplicates(subset=["Duration","Frequency","AmpMult"]).reset_index(drop=True)
# reducedLhx6Data_IntxLR

,Cell Type,Duration,Frequency,AmpMult,ModIn,Averages,predictions_SingleLR,predictions_IntxLR
0,Lhx6,50,100,2.00,-1.000000,-1.000000,-0.723285,-0.795850
1,Lhx6,5000,50,1.00,-1.000000,-0.757576,-0.138198,-0.180481
2,Lhx6,500,20,2.50,0.333333,0.333333,-0.597218,-0.597172
3,Lhx6,1000,10,0.50,0.037037,0.037037,-0.185405,-0.217995
4,Lhx6,1000,200,2.00,-0.875000,-0.832237,-0.529741,-0.563946
...,...,...,...,...,...,...,...,...
114,Lhx6,50,10,1.50,0.311475,0.311475,-0.614443,-0.563932
115,Lhx6,30000,100,1.00,-0.250000,-0.250000,-0.034224,-0.069409
116,Lhx6,1000,150,2.50,0.098039,0.098039,-0.614574,-0.670887
117,Lhx6,250,200,1.00,-1.000000,-1.000000,-0.478014,-0.454737


In [35]:
# #Calculate Pearson and Spearman coefficients
# print("Pearson's correlation value and p-value for LOOXV Lhx6 data interaction terms:")
# print(pearsonr(reducedLhx6Data_IntxLR["predictions_IntxLR"],reducedLhx6Data_IntxLR["Averages"]))
# print(spearmanr(reducedLhx6Data_IntxLR["predictions_IntxLR"],reducedLhx6Data_IntxLR["Averages"]))
# print("\n")

Pearson's correlation value and p-value for LOOXV Lhx6 data interaction terms:
(0.3081958307591513, 0.0006497972596935521)
SpearmanrResult(correlation=0.3250461732309349, pvalue=0.0003099214721077697)


